In [343]:
import pandas as pd
from dotenv import load_dotenv
import os
import json

import requests
import http.client, urllib.parse
from opencage.geocoder import OpenCageGeocode


import pandas_gbq
import google.auth
from google.cloud.bigquery import Client

In [106]:
def get_data(file_path):
    '''Return a dataframe from .sql file stored in @file_path'''
    creds, project = google.auth.default()
    client = Client(project,creds)

    with open(os.path.normpath(file_path), "r") as file:
        query = file.read()

    df = client.query(query).to_dataframe()
    return df

In [326]:
df = get_data('distributor_addr.sql')

C:\Users\y0q00wv\Anaconda3\lib\site-packages\google\auth\_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [136]:
df.head()

,DC_ID,ship_node_addr,ship_node_city,ship_node_state,ship_node_zip
0,2957904,5425 Faa Blvd.,Irving,TX,75061
1,2957918,4555 Redlands Avenue,Perris,CA,92610
2,2957916,415 Wildwood Ave,Woburn,MA,01801
3,888880009,6417 Tri-County Parkway,Schertz,TX,78154
4,25340405,3401 Innovative Way,Mesquite,TX,75149


In [346]:
# load api keys from .env

load_dotenv()

POSITIONSTACK_API_KEY = os.getenv('POSITIONSTACK_API_KEY')
CAGE_API_KEY = os.getenv('CAGE_API_KEY')
MAPS_DATA_API_KEY = os.getenv('MAPS_DATA_API_KEY')

In [327]:
df['ship_node_lat'] = None
df['ship_node_lon'] = None
df['geocoding_confidence'] = None

In [329]:
def get_addr_str_maps(street_addr, city, state, zip_code, country='US'):
    '''Returns full address string used in maps_data api call'''
    return f"{street_addr}, {city}, {state} {zip_code}"

def call_api_maps(url, headers, params):
    '''Returns maps_data api call response'''
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code != 200:
        error_msg = f'Failed to fetch data, status code: {response.status_code}'
        raise Exception(error_msg)
    else:
        return response.json()

def call_api_positionstack(conn, params):
    '''Returns response from positionstack api call'''
    conn.request('GET', '/v1/forward?{}'.format(params))
    response = conn.getresponse()
    response_str = response.read().decode('utf-8')
    response_dict = json.loads(response_str)
    
    if not response_dict['data']:
        return None
    else:
        data = response_dict['data'][0]
    return data

In [330]:
get_addr_str_maps(df['ship_node_addr'][1449], df['ship_node_city'][1449], df['ship_node_state'][1449], df['ship_node_zip'][1449])

'78 West Craig Road, Las Vegas, NV 89032'

### positionstack API

In [332]:
conn = http.client.HTTPConnection('api.positionstack.com')

for index, row in df.iterrows():
    # skip ones with lat value filled
#     if row['ship_node_lat'] is not None:
#         continue

    addr_str = get_addr_str_maps(row['ship_node_addr'], row['ship_node_city'], row['ship_node_state'], row['ship_node_zip'])
    params = urllib.parse.urlencode({
        'access_key': API_KEY,
        'query': addr_str,
        'limit': 1
    })
    
    response = call_api_positionstack(conn, params)
    
    if response:
        df.at[index, 'ship_node_lat'] = response['latitude']
        df.at[index, 'ship_node_lon'] = response['longitude']
        df.at[index, 'geocoding_confidence'] = response['confidence']
    
    if index % 100 == 0:
        print(f"Working on {index} row...")

Working on 0 row...
Working on 100 row...
Working on 200 row...
Working on 300 row...
Working on 400 row...
Working on 500 row...
Working on 600 row...
Working on 700 row...
Working on 800 row...
Working on 900 row...
Working on 1000 row...
Working on 1100 row...
Working on 1200 row...
Working on 1300 row...
Working on 1400 row...


In [333]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   DC_ID                 1450 non-null   int64 
 1   ship_node_addr        1446 non-null   object
 2   ship_node_city        1446 non-null   object
 3   ship_node_state       1446 non-null   object
 4   ship_node_zip         1450 non-null   object
 5   ship_node_lat         1448 non-null   object
 6   ship_node_lon         1448 non-null   object
 7   geocoding_confidence  1448 non-null   object
dtypes: int64(1), object(7)
memory usage: 90.8+ KB


In [334]:
df.geocoding_confidence.value_counts()

1.0    892
0.6    527
0.3     13
0.8      9
0.1      6
0.4      1
Name: geocoding_confidence, dtype: int64

### OpenCage API

In [337]:
OCG = OpenCageGeocode(CAGE_API_KEY)
df['geocoding_confidence_cage'] = None

# fill low confidence score ones with lat, lon from opencage

for index, row in df.iterrows():
    # skip ones with high confidence score
    if row['geocoding_confidence'] is not None and row['geocoding_confidence'] >= 0.8:
        continue

    addr_str = get_addr_str_maps(row['ship_node_addr'], row['ship_node_city'], row['ship_node_state'], row['ship_node_zip'])
    response = OCG.geocode(addr_str)
    
    if response:
        df.at[index, 'ship_node_lat'] = response[0]['geometry']['lat']
        df.at[index, 'ship_node_lon'] = response[0]['geometry']['lng']
        df.at[index, 'geocoding_confidence_cage'] = response[0]['confidence']
    
    if index % 100 == 0:
        print(f"Working on {index} row...")

Working on 100 row...
Working on 200 row...
Working on 500 row...
Working on 700 row...
Working on 1100 row...


RateLimitExceededError: Your rate limit has expired. It will reset to 2500 on 2024-04-17T00:00:00

In [338]:
df[((df['geocoding_confidence'].isna()) | (df['geocoding_confidence'] < 0.8)) & 
   ((df['geocoding_confidence_cage'] < 8) | (df['geocoding_confidence_cage'].isna()))]

,DC_ID,ship_node_addr,ship_node_city,ship_node_state,ship_node_zip,ship_node_lat,ship_node_lon,geocoding_confidence,geocoding_confidence_cage
16,1751203,88 Clyde Alexanda Lane,Pooler,GA,31322,32.3361,-81.3178,0.6,4
48,4748259,2000 Farm To Market Rd 85,Ennis,TX,75119,32.3148,-96.6133,0.6,5
68,3843803,201 Ingram Blvd.,Roseburg,OR,97470,43.1491,-123.362,0.6,7
69,6417706,45 Bruswick Ave,Edison,NJ,08817,40.5198,-74.3934,0.6,5
82,4748208,410 W. Trinity Blvd. #200,Grand Prarie,TX,75050,32.7561,-97.0089,0.3,7
...,...,...,...,...,...,...,...,...,...
1436,628326266,4831 EISENHAUER RD Suite 905,SAN ANTONIO,TX,78218,29.375,-98.5886,0.6,None
1437,628326574,1050 N. Vineyard Ave,Ontario,CA,91764,51.4514,-85.836,0.3,None
1438,4748288,23144 Rd 6 SE,Warden,WA,98857,46.9708,-119.052,0.6,None
1444,628326278,555 CHARCOT AVE # 559,SAN JOSE,CA,95131,37.3546,-121.884,0.6,None


### Maps Data API

In [347]:
# fill low confidence score ones with lat, lon from maps data

url = "https://maps-data.p.rapidapi.com/geocoding.php"

headers = {
    'X-RapidAPI-Key': MAPS_DATA_API_KEY,
    'X-RapidAPI-Host': 'maps-data.p.rapidapi.com'
}

for index, row in df.iterrows():
    # skip ones with high confidence score
    if row['geocoding_confidence'] is not None and row['geocoding_confidence'] >= 0.8:
        continue
        
    if row['geocoding_confidence_cage'] is not None and row['geocoding_confidence_cage'] >= 8:
        continue

    addr_str = get_addr_str_maps(row['ship_node_addr'], row['ship_node_city'], row['ship_node_state'], row['ship_node_zip'])
    querystring = {'query': addr_str, 'lang':'en', 'country':'us'}
    
    response = call_api_maps(url, headers, querystring)
    
    if response:
        df.at[index, 'ship_node_lat'] = response['data']['lat']
        df.at[index, 'ship_node_lon'] = response['data']['lng']
    
    if index % 100 == 0:
        print(f"Working on {index} row...")

Working on 100 row...


In [348]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   DC_ID                      1450 non-null   int64 
 1   ship_node_addr             1446 non-null   object
 2   ship_node_city             1446 non-null   object
 3   ship_node_state            1446 non-null   object
 4   ship_node_zip              1450 non-null   object
 5   ship_node_lat              1445 non-null   object
 6   ship_node_lon              1445 non-null   object
 7   geocoding_confidence       1448 non-null   object
 8   geocoding_confidence_cage  521 non-null    object
dtypes: int64(1), object(8)
memory usage: 102.1+ KB


In [353]:
df[df.columns[:-2]].to_gbq('RogerQin.DSV_DCLocationInfo', project_id='wmt-tebi', if_exists='replace')

1it [00:10, 10.03s/it]
